In [17]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import password
from config import protocol
from config import username
from config import host
from config import host
from config import port
from config import database_name


# National & State Summary - ETL

### Data to populate 5 charts comparing sociodemographic factors between US Census Tracts that are food insecure or not based on the LAhalfand10 flag: 
1. % low income households
2. % non-productive age groups (minors & seniors) by population
3. % race/ethnicity distribution by population
4. % households HUNV
5. % SNAP housing units

[documentation](https://www.ers.usda.gov/data-products/food-access-research-atlas/documentation/)


In [2]:
# Read the food access data from the S3 bucket into a DataFrame
df = pd.read_csv("https://gtbootcamp20230221.s3.amazonaws.com/FoodAccessResearchAtlasData2019.csv", dtype={'CensusTract': str})
df.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,01001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,01001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,01001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,01001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,01001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [3]:
# create state FIP column for mapping
df['StateFIPS'] = df['CensusTract'].str.slice(0,2)
df.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,StateFIPS
0,01001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0,01
1,01001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0,01
2,01001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0,01
3,01001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0,01
4,01001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0,01


In [4]:
# show names of columns
print(df.columns.tolist())

['CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010', 'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle', 'HUNVFlag', 'LowIncomeTracts', 'PovertyRate', 'MedianFamilyIncome', 'LA1and10', 'LAhalfand10', 'LA1and20', 'LATracts_half', 'LATracts1', 'LATracts10', 'LATracts20', 'LATractsVehicle_20', 'LAPOP1_10', 'LAPOP05_10', 'LAPOP1_20', 'LALOWI1_10', 'LALOWI05_10', 'LALOWI1_20', 'lapophalf', 'lapophalfshare', 'lalowihalf', 'lalowihalfshare', 'lakidshalf', 'lakidshalfshare', 'laseniorshalf', 'laseniorshalfshare', 'lawhitehalf', 'lawhitehalfshare', 'lablackhalf', 'lablackhalfshare', 'laasianhalf', 'laasianhalfshare', 'lanhopihalf', 'lanhopihalfshare', 'laaianhalf', 'laaianhalfshare', 'laomultirhalf', 'laomultirhalfshare', 'lahisphalf', 'lahisphalfshare', 'lahunvhalf', 'lahunvhalfshare', 'lasnaphalf', 'lasnaphalfshare', 'lapop1', 'lapop1share', 'lalowi1', 'lalowi1share', 'lakids1', 'lakids1share', 'lase

In [5]:
# summarize total number of population or households in selected features by STATE and LAhalfand10 (flag for low food access)
state_summary_df = df.groupby(['State','LAhalfand10']).agg({
    'Pop2010': 'sum',
    'OHU2010': 'sum',
    'TractLOWI': 'sum',
    'TractKids': 'sum',
    'TractSeniors': 'sum',
    'TractWhite': 'sum',
    'TractBlack': 'sum',
    'TractAsian': 'sum',
    'TractNHOPI': 'sum',
    'TractAIAN': 'sum',
    'TractOMultir': 'sum',
    'TractHispanic': 'sum',
    'TractHUNV': 'sum',
    'TractSNAP': 'sum'
})
# reset index; transform State and LAhalfand10 from index fields to columns
state_summary_df.reset_index(inplace=True)

# change column name
state_summary_df.rename(columns={'State':'Jurisdiction'}, inplace=True)

# display dataframe
state_summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,Alabama,0,2011849,781527,750829.0,477968.0,285743.0,1596873.0,321048.0,13113.0,1094.0,15662.0,64059.0,66205.0,38773.0,104751.0
1,Alabama,1,2767887,1102264,1042096.0,654491.0,372049.0,1678521.0,930263.0,40482.0,1963.0,12556.0,104102.0,119397.0,76587.0,164852.0
2,Alaska,0,186563,67331,42372.0,46209.0,13900.0,136745.0,2989.0,5396.0,735.0,18974.0,14265.0,7935.0,4215.0,5371.0
3,Alaska,1,523668,190727,134581.0,138068.0,40634.0,336630.0,20273.0,32721.0,6674.0,78812.0,48558.0,31307.0,18259.0,20484.0
4,Arizona,0,857778,321996,346581.0,209192.0,117376.0,617568.0,31972.0,19973.0,2419.0,60861.0,124985.0,245099.0,24916.0,43053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,West Virginia,1,921047,386789,345703.0,189943.0,150293.0,846547.0,42880.0,8959.0,289.0,2015.0,20357.0,13740.0,34875.0,60530.0
98,Wisconsin,0,1953902,765486,532677.0,460000.0,267095.0,1740172.0,93087.0,23524.0,561.0,23374.0,73184.0,107933.0,48700.0,81651.0
99,Wisconsin,1,3733084,1514282,1059719.0,879492.0,510219.0,3161895.0,266061.0,105710.0,1266.0,31152.0,167000.0,228123.0,109439.0,187980.0
100,Wyoming,0,137770,52261,39444.0,36212.0,15520.0,120061.0,640.0,756.0,103.0,8315.0,7895.0,13073.0,1580.0,3681.0


In [6]:
# summarize total number of population or households in selected features for US by LAhalfand10 (flag for low food access)
us_summary_df = df.groupby(['LAhalfand10']).agg({
    'Pop2010': 'sum',
    'OHU2010': 'sum',
    'TractLOWI': 'sum',
    'TractKids': 'sum',
    'TractSeniors': 'sum',
    'TractWhite': 'sum',
    'TractBlack': 'sum',
    'TractAsian': 'sum',
    'TractNHOPI': 'sum',
    'TractAIAN': 'sum',
    'TractOMultir': 'sum',
    'TractHispanic': 'sum',
    'TractHUNV': 'sum',
    'TractSNAP': 'sum'
})
# reset index; transform LAhalfand10 from index field to column
us_summary_df.reset_index(inplace=True)

# add Jurisdiction column
us_summary_df['Jurisdiction'] = 'United States'

# display data
us_summary_df


,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Jurisdiction
0,0,91132485,34733952,31407468.0,21215438.0,12063938.0,67587011.0,10152811.0,3909156.0,129476.0,943163.0,8389823.0,14436336.0,4542964.0,4715999.0,United States
1,1,217613053,81982340,69046369.0,52957586.0,28202843.0,155965656.0,28776503.0,10765064.0,410535.0,1968964.0,19726331.0,36040953.0,5879872.0,9916554.0,United States


In [7]:
# concatenate state and US total data
summary_df = pd.concat([state_summary_df,us_summary_df])
summary_df.reset_index(inplace=True, drop=True)
summary_df.tail()

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
99,Wisconsin,1,3733084,1514282,1059719.0,879492.0,510219.0,3161895.0,266061.0,105710.0,1266.0,31152.0,167000.0,228123.0,109439.0,187980.0
100,Wyoming,0,137770,52261,39444.0,36212.0,15520.0,120061.0,640.0,756.0,103.0,8315.0,7895.0,13073.0,1580.0,3681.0
101,Wyoming,1,425856,174618,117110.0,99190.0,54570.0,391218.0,4108.0,3670.0,324.0,5021.0,21515.0,37158.0,7293.0,9558.0
102,United States,0,91132485,34733952,31407468.0,21215438.0,12063938.0,67587011.0,10152811.0,3909156.0,129476.0,943163.0,8389823.0,14436336.0,4542964.0,4715999.0
103,United States,1,217613053,81982340,69046369.0,52957586.0,28202843.0,155965656.0,28776503.0,10765064.0,410535.0,1968964.0,19726331.0,36040953.0,5879872.0,9916554.0


## Calculate percentages from totals and add to summary_df
1. % low income Population 
    - TractLOWI/Pop2010
2. % non-productive age groups (minors & seniors) by population
    - TractKids/Pop2010
    - TractSeniors/Pop2010
3. % race/ethnicity distribution by population
    - TractWhite/Pop2010
    - TractBlack/Pop2010
    - TractAsian/Pop2010
    - TractNHOPI/Pop2010
    - TractAIAN/Pop2010
    - TractOMultir/Pop2010
    - TractHispanic/Pop2010
4. % households HUNV
    - TractHUNV/OHU2010
5. % SNAP housing units
    - TractSNAP/OHU2010

# 1st Visual

In [8]:
# % low income distribution by population 
summary_df['LOWI_per'] = summary_df['TractLOWI']/summary_df['Pop2010']
summary_df = summary_df.sort_values('LOWI_per', ascending=False)
summary_df.reset_index(inplace=True, drop=True)

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,LOWI_per
0,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,938.0,6165.0,51948.0,58855.0,51598.0,127056.0,0.428383
1,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,3027.0,6240.0,110569.0,179904.0,20235.0,31652.0,0.424458
2,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,4921.0,13480.0,116876.0,142552.0,54545.0,100009.0,0.415398
3,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,1621.0,13589.0,181309.0,835730.0,103946.0,227394.0,0.412185
4,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,1567.0,157358.0,325660.0,802015.0,37610.0,109229.0,0.408970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,1029.0,8009.0,209607.0,346393.0,87111.0,120605.0,0.212779
100,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,92751.0,2822.0,243913.0,86407.0,16257.0,30626.0,0.211106
101,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,215.0,1724.0,21832.0,25371.0,16884.0,22227.0,0.201444
102,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,169.0,1426.0,11612.0,11333.0,10416.0,15832.0,0.194524


# 2nd Visual

In [9]:
# % non-productive age groups (minors & seniors) by population
kids_seniors = summary_df['TractKids'] + summary_df['TractSeniors']

summary_df['Non_prod_ages_per'] = kids_seniors/summary_df['Pop2010']
summary_df.reset_index(inplace=True, drop=True)

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,LOWI_per,Non_prod_ages_per
0,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,938.0,6165.0,51948.0,58855.0,51598.0,127056.0,0.428383,0.381782
1,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,3027.0,6240.0,110569.0,179904.0,20235.0,31652.0,0.424458,0.372707
2,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,4921.0,13480.0,116876.0,142552.0,54545.0,100009.0,0.415398,0.385710
3,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,1621.0,13589.0,181309.0,835730.0,103946.0,227394.0,0.412185,0.380205
4,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,1567.0,157358.0,325660.0,802015.0,37610.0,109229.0,0.408970,0.385335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,1029.0,8009.0,209607.0,346393.0,87111.0,120605.0,0.212779,0.372604
100,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,92751.0,2822.0,243913.0,86407.0,16257.0,30626.0,0.211106,0.373548
101,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,215.0,1724.0,21832.0,25371.0,16884.0,22227.0,0.201444,0.352402
102,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,169.0,1426.0,11612.0,11333.0,10416.0,15832.0,0.194524,0.355037


# 3rd Visual

In [10]:
# % race/ethnicity distribution by population
summary_df['White_per'] = summary_df['TractWhite']/summary_df['Pop2010']
summary_df['Black_per'] = summary_df['TractBlack']/summary_df['Pop2010']
summary_df['Asian_per'] = summary_df['TractAsian']/summary_df['Pop2010']
summary_df['NHOPI_per'] = summary_df['TractNHOPI']/summary_df['Pop2010']
summary_df['AIAN_per'] = summary_df['TractAIAN']/summary_df['Pop2010']
summary_df['Multi_Race_per'] = summary_df['TractOMultir']/summary_df['Pop2010']
summary_df['Hispanic_per'] = summary_df['TractHispanic']/summary_df['Pop2010']
summary_df.reset_index(inplace=True, drop=True)

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,TractSNAP,LOWI_per,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per
0,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,127056.0,0.428383,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155
1,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,31652.0,0.424458,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634
2,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,100009.0,0.415398,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994
3,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,227394.0,0.412185,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106
4,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,109229.0,0.408970,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,120605.0,0.212779,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025
100,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,30626.0,0.211106,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740
101,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,22227.0,0.201444,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312
102,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,15832.0,0.194524,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952


# 4th Visual

In [11]:
#% households HUNV distribution by housing units
summary_df['HUNV_per'] = summary_df['TractHUNV']/summary_df['OHU2010']
summary_df.reset_index(inplace=True, drop=True)

summary_df

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,LOWI_per,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per,HUNV_per
0,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,0.428383,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155,0.070196
1,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,0.424458,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634,0.107713
2,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,0.415398,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994,0.072633
3,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,0.412185,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106,0.083852
4,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,0.408970,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298,0.058190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,0.212779,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025,0.081795
100,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,0.211106,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740,0.053943
101,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,0.201444,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312,0.059442
102,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,0.194524,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952,0.044337


# 5th Visual

In [12]:
#% SNAP housing units by housing units
summary_df['SNAP_per'] = summary_df['TractSNAP']/summary_df['OHU2010']
summary_df.reset_index(inplace=True, drop=True)

summary_df


,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per,HUNV_per,SNAP_per
0,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155,0.070196,0.172852
1,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634,0.107713,0.168486
2,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994,0.072633,0.133174
3,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106,0.083852,0.183435
4,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298,0.058190,0.169000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Connecticut,1,2770589,1064989,589524.0,626041.0,406291.0,2159021.0,280554.0,112369.0,...,0.372604,0.779264,0.101262,0.040558,0.000371,0.002891,0.075654,0.125025,0.081795,0.113245
100,Hawaii,1,952246,301372,201025.0,221184.0,134526.0,231630.0,16555.0,364575.0,...,0.373548,0.243246,0.017385,0.382858,0.097402,0.002964,0.256145,0.090740,0.053943,0.101622
101,New Hampshire,1,718473,284043,144732.0,156100.0,97091.0,663837.0,10621.0,20244.0,...,0.352402,0.923955,0.014783,0.028176,0.000299,0.002400,0.030387,0.035312,0.059442,0.078252
102,New Hampshire,0,597997,234930,116325.0,131134.0,81177.0,572213.0,4414.0,8163.0,...,0.355037,0.956883,0.007381,0.013651,0.000283,0.002385,0.019418,0.018952,0.044337,0.067390


In [13]:
# Get df information
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Jurisdiction       104 non-null    object 
 1   LAhalfand10        104 non-null    int64  
 2   Pop2010            104 non-null    int64  
 3   OHU2010            104 non-null    int64  
 4   TractLOWI          104 non-null    float64
 5   TractKids          104 non-null    float64
 6   TractSeniors       104 non-null    float64
 7   TractWhite         104 non-null    float64
 8   TractBlack         104 non-null    float64
 9   TractAsian         104 non-null    float64
 10  TractNHOPI         104 non-null    float64
 11  TractAIAN          104 non-null    float64
 12  TractOMultir       104 non-null    float64
 13  TractHispanic      104 non-null    float64
 14  TractHUNV          104 non-null    float64
 15  TractSNAP          104 non-null    float64
 16  LOWI_per           104 non

In [14]:
# Check for null values in df
summary_df.isnull().sum()

Jurisdiction         0
LAhalfand10          0
Pop2010              0
OHU2010              0
TractLOWI            0
TractKids            0
TractSeniors         0
TractWhite           0
TractBlack           0
TractAsian           0
TractNHOPI           0
TractAIAN            0
TractOMultir         0
TractHispanic        0
TractHUNV            0
TractSNAP            0
LOWI_per             0
Non_prod_ages_per    0
White_per            0
Black_per            0
Asian_per            0
NHOPI_per            0
AIAN_per             0
Multi_Race_per       0
Hispanic_per         0
HUNV_per             0
SNAP_per             0
dtype: int64

In [15]:
# Check for duplicate values in the df 
summary_df.duplicated().sum()

0

In [16]:
summary_df.to_csv("Summary.csv")

#### Summary dataframe exported to Postgres for table visualizations. Though TableauPublic has limitation and need liscnenced version to setup connection.

In [18]:
# Connection to PostgresSQL
protocol = protocol
username = username
password = password
host = host
port = port
database_name = database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [19]:
# Confirm tables names
engine.table_names()

['food_access', 'summary']

In [21]:
# Export the transformed data into the PostgresSQL
summary_df.to_sql(name='summary', con=engine, if_exists='replace', index=False)

In [22]:
# Read the data from Postgres table
pd.read_sql_query('select * from summary', con=engine).head()

,Jurisdiction,LAhalfand10,Pop2010,OHU2010,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,...,Non_prod_ages_per,White_per,Black_per,Asian_per,NHOPI_per,AIAN_per,Multi_Race_per,Hispanic_per,HUNV_per,SNAP_per
0,Mississippi,1,1951759,735055,836101.0,498648.0,246498.0,1064463.0,808634.0,19611.0,...,0.381782,0.545386,0.414310,0.010048,0.000481,0.003159,0.026616,0.030155,0.070196,0.172852
1,Nevada,0,513082,187861,217782.0,130994.0,60235.0,327664.0,35398.0,30184.0,...,0.372707,0.638619,0.068991,0.058829,0.005900,0.012162,0.215500,0.350634,0.107713,0.168486
2,Arkansas,1,1900847,750965,789608.0,465127.0,268048.0,1368963.0,369367.0,27240.0,...,0.385710,0.720186,0.194317,0.014330,0.002589,0.007092,0.061486,0.074994,0.072633,0.133174
3,Florida,0,3225435,1239642,1329476.0,661500.0,564828.0,2508728.0,474426.0,45762.0,...,0.380205,0.777795,0.147089,0.014188,0.000503,0.004213,0.056212,0.259106,0.083852,0.183435
4,New Mexico,1,1694526,646327,693011.0,432734.0,220226.0,1147476.0,37524.0,24941.0,...,0.385335,0.677166,0.022144,0.014719,0.000925,0.092863,0.192184,0.473298,0.058190,0.169000
